In [1]:
import os
import keras
from keras.preprocessing.image import load_img,img_to_array
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from sklearn import preprocessing

import numpy as np
import pandas as pd
from glob import glob
import joblib
import cv2

2022-03-23 20:41:44.881451: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-23 20:41:44.881480: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
model_path = '/home/vimalkumar/Documents/Python_new_1/SOFTCO/New project'



ineption_angle_model = keras.models.load_model(os.path.join(model_path,
                                            'stewart_rotation_inception_model.h5'))
vgg16_angle_model = keras.models.load_model(os.path.join(model_path,
                                            'vgg16_stewart_rotation.h5'))

vggxgb_model = keras.models.load_model('vgg16_xgb.h5')
xgb_model = joblib.load('xgb.pkl')

vgg19_angle_model = keras.models.load_model(os.path.join(model_path,
                                            'stewart_vgg19_rotation.h5'))


2022-03-23 20:42:03.083304: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-23 20:42:03.083344: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-23 20:42:03.083368: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (IG154): /proc/driver/nvidia/version does not exist
2022-03-23 20:42:03.084074: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
dic = {0: '0 degree', 1: '180 degree', 2: '270 degree', 3: '90 degree'}
print(dic)

{0: '0 degree', 1: '180 degree', 2: '270 degree', 3: '90 degree'}


In [20]:
path = '/home/vimalkumar/Downloads/samples/batch_10_/batch_10/test'

inception_result = []
vgg16_result = []
xgb_result = []
vgg19_result = []
filename = []

cnt = 1
for i in os.listdir(path):
    print(cnt , '  ',i)
    filename.append(i)
    cnt += 1
    img = image.load_img(os.path.join(path,i), 
                         target_size=(224,224))
#     plt.imshow(img)
#     plt.show()

    x_in = image.img_to_array(img)
    x_in = x_in/255
    x_in = np.expand_dims(x_in,axis=0)
    images_in = np.vstack([x_in])
    val_in = ineption_angle_model.predict(images_in)
    #print(val)

    a_in=np.argmax(val_in,axis=1)
    #print(a)
    #print('Rotation angle : ',dic[a_in[0]])
    inception_result.append(dic[a_in[0]])
    
    x_vgg16 = image.img_to_array(img)
    x_vgg16 = x_vgg16/255
    x_vgg16 = np.expand_dims(x_vgg16,axis=0)
    images_vgg16 = np.vstack([x_vgg16])
    val_vgg16 = vgg16_angle_model.predict(images_vgg16)
    #print(val)

    a_vgg16=np.argmax(val_vgg16,axis=1)
    #print(a)
    #print('Rotation angle : ',dic[a_vgg[0]])
    vgg16_result.append(dic[a_vgg16[0]])
    
    x_vgg19 = image.img_to_array(img)
    x_vgg19 = x_vgg19/255
    x_vgg19 = np.expand_dims(x_vgg19,axis=0)
    images_vgg19 = np.vstack([x_vgg19])
    val_vgg19 = vgg19_angle_model.predict(images_vgg19)
    #print(val)

    a_vgg19=np.argmax(val_vgg19,axis=1)
    #print(a)
    #print('Rotation angle : ',dic[a_vgg[0]])
    vgg19_result.append(dic[a_vgg19[0]])
    
    
    imm=cv2.imread(os.path.join(path,i))
    imm=cv2.resize(imm,(256,256))
#     img_1=img/255.0
    
    input_img = np.expand_dims(imm, axis=0) #Expand dims so the input is (num images, x, y, c)
    input_img_feature=vggxgb_model.predict(input_img)
    input_img_features=input_img_feature.reshape(input_img_feature.shape[0], -1)
    prediction = xgb_model.predict(input_img_features)[0]
    #print(dic[prediction])
    #prediction = le.inverse_transform([prediction]) #Reverse the label encoder to original name
    #print(name, prediction)
    xgb_result.append(dic[prediction])
    
df = pd.DataFrame({'file':filename,'inception':inception_result,'vgg':vgg16_result,'xgb':xgb_result,
                  'vgg19':vgg19_result})
pd.set_option('display.max_rows',1000)
df

1    245.png
2    173.png
3    11.png
4    268.png
5    33.png
6    210.png
7    166.png
8    87.png
9    19.png
10    222.png
11    169.png
12    45.png
13    149.png
14    104.png
15    75.png
16    99.png
17    37.png
18    7.png
19    6.png
20    96.png
21    196.png
22    14.png
23    175.png
24    135.png


,file,inception,vgg,xgb,vgg19
0,245.png,0 degree,0 degree,180 degree,0 degree
1,173.png,180 degree,180 degree,0 degree,180 degree
2,11.png,0 degree,180 degree,0 degree,180 degree
3,268.png,180 degree,0 degree,180 degree,0 degree
4,33.png,90 degree,0 degree,270 degree,0 degree
5,210.png,180 degree,180 degree,0 degree,180 degree
6,166.png,180 degree,180 degree,0 degree,0 degree
7,87.png,180 degree,180 degree,180 degree,0 degree
8,19.png,180 degree,0 degree,0 degree,0 degree
9,222.png,0 degree,0 degree,0 degree,0 degree


In [21]:
df.head()
x, *y = ['inception', 'vgg', 'xgb','vgg19']
df['confidence'] = df[y].eq(df[x], axis=0).all(1)
df[df['confidence']==True]

,file,inception,vgg,xgb,vgg19,confidence
9,222.png,0 degree,0 degree,0 degree,0 degree,True
11,45.png,90 degree,90 degree,90 degree,90 degree,True
13,104.png,0 degree,0 degree,0 degree,0 degree,True
15,99.png,0 degree,0 degree,0 degree,0 degree,True
20,196.png,180 degree,180 degree,180 degree,180 degree,True
22,175.png,0 degree,0 degree,0 degree,0 degree,True
23,135.png,0 degree,0 degree,0 degree,0 degree,True


In [22]:
df['confidence'].value_counts()

False    17
True      7
Name: confidence, dtype: int64